In [196]:
import pandas as pd
from fuzzywuzzy import process

Este notebook es para pegarle la informacion del snies a los resultados del valor agregado enviados por el icfes. El objetivo es armar el identificador icine, con el fin de comparar los resultados del icfes 2021-2022, con los resultados propios 2022-2023. El resultado se le envió a Julian el 16 de julio para la presentación con victor el 17 de julio. 

Esto lo haremos:

1. pegandole la llave de la bace cine a la base del icves,módulo diferencias de sintaxis
2. Una vez la base del icfes tiene la llave del cine, pegarle la info del cine.
3. construir la columna icine (icine especifico)

In [197]:
cols_base_cine = [
    'codigo_institucion',
    'codigo_institucion_padre',
    'nombre_institucion',
    'estado_institucion',
    'caracter_academico',
    'codigo_snies_del_programa',
    'nombre_del_programa',
    'estado_programa',
    'cine_f_2013_ac_campo_especific',
    'nucleo_basico_del_conocimiento',
    'nivel_academico',
    'municipio_oferta_programa',
    'id_cine_campo_especifico',
    'codigo_del_municipio_programa',
    'codigo_de_la_institucion',
    'institucion_de_educacion_superior_ies',
    'ies_acreditada',
    'caracter_ies',
    'municipio_oferta_programa'
]

In [198]:
cols_icfes = [
    'inst_cod_institucion', 
    'NBC2',
    'INBC',
    'va_lectura_critica',
    'va_razona_cuantitat',
]

In [199]:
cine_nbc_programas_file = "base_cine_programas_vigencia_2025_18072025.xlsx"
#cine_nbc_programas_file = "base_cine_programas_vigencia_2022_2023_15072025.xlsx"
cine_nbc_programas_path = "../../data/SNIES_CINE_cleaned/"

va_icfes_path = "../../data/Resultados_ICFES_VA/"
va_icfes_file = "VA_INBCs_Bogota.xlsx"

In [200]:
va_icfes = pd.read_excel(va_icfes_path + va_icfes_file, usecols = cols_icfes)
cine_nbc_programas = pd.read_excel(cine_nbc_programas_path + cine_nbc_programas_file, usecols = cols_base_cine)

# Elimina duplicados según 'codigo_snies_del_programa', dejando solo la primera aparición
cine_nbc_programas = cine_nbc_programas.drop_duplicates(subset="codigo_snies_del_programa", keep="first")
cine_nbc_programas = cine_nbc_programas.drop_duplicates(subset=["codigo_institucion","nucleo_basico_del_conocimiento"], keep="first")

cine_nbc_programas['periodo'] = '2021-2022'

cine_nbc_programas['nucleo_basico_del_conocimiento_mayus'] = (
    cine_nbc_programas['nucleo_basico_del_conocimiento'].str.upper()
)


In [201]:
print(len(cine_nbc_programas['nucleo_basico_del_conocimiento_mayus'].sort_values().unique()))
print(len(va_icfes["NBC2"].sort_values().unique()))

56
48


In [202]:
# Listas únicas
nbc_1 = cine_nbc_programas['nucleo_basico_del_conocimiento_mayus'].dropna().unique()
nbc_2 = va_icfes["NBC2"].dropna().unique()

# Empatar cada item de nbc_2 con el más similar en nbc_1
matches = []
for item in nbc_2:
    match, score = process.extractOne(item, nbc_1)
    matches.append((item, match, score))

# Convertir a DataFrame para ver el resultado
fuzzy_matches = pd.DataFrame(matches, columns=["NBC2", "NBC_cine", "similarity_score"])

# Filtrar los que tengan score alto (>90 por ejemplo)
fuzzy_matches_high = fuzzy_matches[fuzzy_matches["similarity_score"] >= 90]

# Quedarse solo con la mejor coincidencia por cada NBC2
fuzzy_matches_high_unique = fuzzy_matches_high.sort_values('similarity_score', ascending=False).drop_duplicates('NBC2')

# Unir la información de los matches a la base ICFES
va_icfes_merged = va_icfes.merge(
    fuzzy_matches_high_unique[["NBC2", "NBC_cine"]],
    on="NBC2",
    how="inner"
)



Ahora va_icfes_merged tiene la informacion original mas la columna que permtira pegar con la base del sines

Mirar cuales nbc quedaron por fuera

In [203]:
#✅ Paso 1: Asegúrate de eliminar NaN y duplicados
nbc2_unique = va_icfes["NBC2"].dropna().unique()
matched_nbc2 = fuzzy_matches_high["NBC2"].dropna().unique()  # Cambiado a "NBC2"

#✅ Paso 2: Verifica si todos están presentes
all_matched = set(nbc2_unique).issubset(set(matched_nbc2))
print("¿Todos los valores de NBC2 están en fuzzy_matches_high?", all_matched)

# Paso 3 (opcional): Ver qué valores faltan
missing_nbc2 = set(nbc2_unique) - set(matched_nbc2)
print("Valores de NBC2 que no fueron emparejados:")
for val in sorted(missing_nbc2):
    print("-", val)

¿Todos los valores de NBC2 están en fuzzy_matches_high? True
Valores de NBC2 que no fueron emparejados:


Enriquecemos el dataframe del icfes con la información del snies

In [245]:
va_icfes_enriquecido = va_icfes_merged.merge(
    cine_nbc_programas,
    left_on=["inst_cod_institucion","NBC_cine"],
    right_on=["codigo_institucion","nucleo_basico_del_conocimiento_mayus"],
    indicator=True,
    how = "left"
)

va_icfes_enriquecido["icine_especifico"] = (
    va_icfes_enriquecido["inst_cod_institucion"].astype(str) + "_" +
    va_icfes_enriquecido["cine_f_2013_ac_campo_especific"].astype(str)
)

va_icfes_enriquecido['periodo'] = "2021-2022"
va_icfes_enriquecido.codigo_snies_del_programa = va_icfes_enriquecido.codigo_snies_del_programa.astype("Int64")

#va_icfes_enriquecido = va_icfes_enriquecido.drop_duplicates(subset=["inst_cod_institucion", "NBC_cine"], keep="first")

In [246]:
# Contar cuántas filas tienen _merge == "both"
num_both = (va_icfes_enriquecido["_merge"] == "both").sum()
print(f"Número de filas con _merge == 'both': {num_both}")

Número de filas con _merge == 'both': 440


In [247]:
va_icfes_enriquecido = va_icfes_enriquecido.rename(columns={
    'icine_especifico': 'icine',
    'nombre_del_programa': 'nombres_programas',
    'codigo_snies_del_programa': 'snies_programas',
    'va_lectura_critica': 'coeficiente_LC',
    'va_razona_cuantitat': 'coeficiente_RC'
})

campos_exportar = [
    'icine',
    'codigo_institucion',
    'snies_programas',
    'nombres_programas',
    'nombre_institucion',
    'municipio_oferta_programa',
    'coeficiente_LC',
    'coeficiente_RC',
    "periodo",
    "_merge"
]


va_icfes_enriquecido = va_icfes_enriquecido[campos_exportar]


va_icfes_enriquecido.reset_index(drop=True).to_excel("../../data/Resultados_ICFES_VA/VA_INBCS_ICINE_BOGOTA.xlsx",index=False)

#va_icfes_enriquecido[campos_exportar].reset_index(drop=True).to_excel("../../data/Resultados_ICFES_VA/VA_INBCS_ICINE_BOGOTA_2022_2023.xlsx",index=False)

In [248]:
va_icfes_enriquecido.shape

(458, 10)

mirar cuantos programas aperecerían tendrían más de 2 icines

Ahora concatenamos los resultados propios con los del snies

In [300]:

va_icfes_path = "../../data/Resultados_ICFES_VA/"
va_icfes_file = "VA_INBCS_ICINE_Bogota.xlsx"

va_atenea_path = "../../data/Resultados_VA/cine_especifico/bogota_region/"
va_atenea_file = "va_cine_especifico_2021_2022_nacionales_run18072025.xlsx"

In [301]:
va_icfes_enriquecido.columns

Index(['icine', 'codigo_institucion', 'snies_programas', 'nombres_programas',
       'nombre_institucion', 'municipio_oferta_programa', 'coeficiente_LC',
       'coeficiente_RC', 'periodo', '_merge'],
      dtype='object')

In [ ]:
# Leer los archivos
va_icfes = pd.read_excel(va_icfes_path + va_icfes_file)
va_atenea = pd.read_excel(va_atenea_path + va_atenea_file)

# Agregar columna de origen
va_icfes['origen'] = 'icfes'
va_atenea['origen'] = 'atenea'

# Asegurarse de que va_atenea tenga la columna _merge
if '_merge' not in va_atenea.columns:
    va_atenea['_merge'] = 'no_tener_en_cuenta_este_valor'

# Igualar columnas y orden (opcional, pero recomendable)
columnas_interes = va_icfes.columns
va_atenea = va_atenea[columnas_interes]

# Concatenar ambos DataFrames
df_concatenado = pd.concat([va_icfes, va_atenea], ignore_index=True)

# Identificar los icine que están en ambas bases
conteo_icine = df_concatenado.groupby('icine')['origen'].nunique()
icine_en_ambas = conteo_icine[conteo_icine == 2].index
df_concatenado['icine_en_ambas'] = df_concatenado['icine'].isin(icine_en_ambas)

In [308]:
df_concatenado.head()

,icine,codigo_institucion,snies_programas,nombres_programas,nombre_institucion,municipio_oferta_programa,coeficiente_LC,coeficiente_RC,periodo,_merge,origen,icine_en_ambas,programa_en_ambas
0,1101_Arquitectura y construcción,1101.0,30.0,Arquitectura,Universidad Nacional De Colombia,bogota_dc,7.618661,5.031926,2021-2022,both,icfes,True,False
1,1101_Salud,1101.0,9.0,Medicina,Universidad Nacional De Colombia,bogota_dc,8.087526,11.509295,2021-2022,both,icfes,True,False
2,1101_Ingeniería y profesiones afines,1101.0,29.0,Ingenieria Quimica,Universidad Nacional De Colombia,bogota_dc,3.061149,8.302320,2021-2022,both,icfes,True,False
3,1101_Artes,1101.0,5.0,Diseno Industrial,Universidad Nacional De Colombia,bogota_dc,2.000692,4.361184,2021-2022,both,icfes,True,False
4,1101_Humanidades (Excepto idiomas),1101.0,13.0,Antropologia,Universidad Nacional De Colombia,bogota_dc,3.803282,2.057371,2021-2022,both,icfes,True,False


In [310]:
df_concatenado.to_excel(r'../../data/Resultados_ICFES_VA/va_atenea_icfes_2021_2022_info_snies_2021-2022.xlsx', index=False)

Comparar con cual informacion del snies es mejor hacer el ejercicio
- Si con snies 2021-2022
- Si con snies 2025

In [314]:
import pandas as pd

va_icfes_path = "../../data/Resultados_ICFES_VA/"
va_icfes_file_pasado = "va_atenea_icfes_2021_2022_info_snies_2021-2022.xlsx"
va_icfes_file_presente = "va_atenea_icfes_2021_2022_info_snies_2025.xlsx"

# Leer los archivos
df_pasado = pd.read_excel(va_icfes_path + va_icfes_file_pasado)
df_presente = pd.read_excel(va_icfes_path + va_icfes_file_presente)

In [316]:
import pandas as pd

def resumen_icines(df):
    icine_icfes = set(df.loc[df['origen'] == 'icfes', 'icine'].unique())
    icine_atenea = set(df.loc[df['origen'] == 'atenea', 'icine'].unique())
    solo_icfes = icine_icfes - icine_atenea
    solo_atenea = icine_atenea - icine_icfes
    en_ambos = icine_icfes & icine_atenea
    return pd.DataFrame([{
        'solo_icfes': len(solo_icfes),
        'solo_atenea': len(solo_atenea),
        'en_ambos': len(en_ambos)
    }])

# Tabla para df_pasado
tabla_pasado = resumen_icines(df_pasado)
print("ICINES cuando la info es del snies 2021-2022:")
print(tabla_pasado)

# Tabla para df_presente
tabla_presente = resumen_icines(df_presente)
print("\nICINES cuando la info es del snies 2025:")
print(tabla_presente)

ICINES cuando la info es del snies 2021-2022:
   solo_icfes  solo_atenea  en_ambos
0         105          489       175

ICINES cuando la info es del snies 2025:
   solo_icfes  solo_atenea  en_ambos
0          52          714       228
